In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
standings_url="https://fbref.com/en/comps/20/2016-2017/2016-2017-Bundesliga-Stats"
years = list(range(2017, 2013, -1))
all_matches = []

In [2]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
        Scores_Fixtures = Scores_Fixtures[['Date','Time','Comp','Round','Day','Venue','GF','GA','Opponent','Poss']]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/defense/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        defense = pd.read_html(data.text, match="Defensive Actions")[0]
        defense.columns = defense.columns.droplevel()
        
        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/keeper/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
        goalkeeping.columns = goalkeeping.columns.droplevel()

        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        passing = pd.read_html(data.text, match="Passing")[0]
        upper_level = passing.columns.get_level_values(0)
        new_columns = [f'{prefix}_{col}' for prefix, col in zip(upper_level, passing.columns.get_level_values(1))]
        passing.columns = new_columns
        passing.rename(columns={passing.columns[0]: 'Date'}, inplace=True)

        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/possession/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        possession = pd.read_html(data.text, match="Possession")[0]
        possession.columns = possession.columns.droplevel()
        
        #try:
        team_data = Scores_Fixtures.merge(shooting[['Date','Sh']], on = ['Date'])
        team_data = team_data.merge(goalkeeping[['Date','Save%']], on = ['Date'])
        team_data = team_data.merge(passing[['Date','Total_Att','Total_Cmp%']], on = ['Date'])
        team_data = team_data.merge(possession[['Date','Att']], on = ['Date'])
        team_data = team_data.merge(defense[['Date','Blocks']], on = ['Date'])
        #except ValueError:
            #continue
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20)

C:\Users\tramn\AppData\Local\Temp\ipykernel_4324\1783270792.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_4324\1783270792.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


IndexError: list index out of range

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("19_23_matches.csv")